In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict
from matplotlib import pyplot as plt
import seaborn as sns
import mplcursors
import math

In [2]:
def init_graph(G, node_adj_frame):
  G.add_nodes_from([i for i in range(len(node_adj_frame))])
  labels = {}
  labels = node_adj_frame.columns
  for i in range(len(node_adj_frame)):
    snode= node_adj_frame[labels[0]][i]-1
    temp = node_adj_frame[labels[2]][i]
    if ',' in str(temp):
      sedge_arr=temp.split(',')
      for j in range(0,len(sedge_arr)):
        k=int(sedge_arr[j])
        if k<81:
          G.add_edge(snode,k-1)  
        else:
          G.add_edge(snode,k-2)
    elif temp==np.nan:
      print("ERROR:None found in th adjacency excel sheet")
    else :
      G.add_edge(snode,int(temp)-1)
  return

In [3]:
def init_graph_attr(G, AdjFile, df_IMR, df_MMR, df_PAW, IMR_Key, MMR_Key, PAW_Key):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["KGISTalukN"].tolist()
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    capability_vector = []
    capability_vector = [df_IMR[IMR_Key], df_MMR[MMR_Key], df_PAW[PAW_Key]]
    node_attri_dict = {}
    for i, key in enumerate(df_IMR["Taluka"]):
        node_attri_dict[key] = [row[i] for row in capability_vector]
    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capability"] = node_attri_dict.get(node_list[i], [0, 0, 0])
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp
    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df_IMR = pd.read_excel('PIA_Normalized_IMR.xlsx')
df_MMR = pd.read_excel('PIA_Normalized_MMR.xlsx')
df_PAW = pd.read_excel('Pregnant_Anaemic_Women_PIA.xlsx')

In [5]:
dictionary_IMR = {"":"", "":"", "":"", "":"", "":"", "":"", "":"", "":"", "":""}
dictionary_MMR = {"":"", "":"", "":"", "":"", "":"", "":"", "":"", "":"", "":""}
dictionary_PAW = {"":"", "":"", "":"", "":"", "":"", "":"", "":"", "":"", "":""}

In [6]:
dist_taluk_dict = defaultdict(list)
for k, v in zip(df_IMR["District_GIS"],df_IMR["Taluka"]):
  dist_taluk_dict[k].append(v)

In [7]:
IMR_Key = 'Normalized IMR'
MMR_Key = 'Normalized MMR'
PAW_Key = 'Normalized %Pregnant women age 15-49 years who are anaemic '

In [8]:
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df_IMR, df_MMR, df_PAW, IMR_Key, MMR_Key, PAW_Key)

In [9]:
def get_node_stress(G):
  taluk_stress_dict={}
  for n in G.nodes():
    centroid = [0, 0, 0]
    temp_dist_imr = 0
    temp_dist_mmr = 0
    temp_dist_paw = 0
    temp_dist = 0
    node_stress = 0
    neigList = list(G.neighbors(n))
    for nei in neigList:
      try:
        temp_dist_imr += G.nodes[nei]['capability'][0]
        temp_dist_mmr += G.nodes[nei]['capability'][1]
        temp_dist_paw += G.nodes[nei]['capability'][2]
        # temp_dist_imr = abs(G.nodes[nei]['capability'][0] - G.nodes[n]['capability'][0])
        # temp_dist_mmr = abs(G.nodes[nei]['capability'][1] - G.nodes[n]['capability'][1])
        # temp_dist = math.sqrt(abs(temp_dist_imr + temp_dist_mmr))
        # node_stress += temp_dist
      except(KeyError):
        pass
    centroid = [temp_dist_imr/len(neigList), temp_dist_mmr/len(neigList), temp_dist_paw/len(neigList)]
    # print(centroid)
    temp_dist_mmr = 0
    temp_dist_imr = 0
    temp_dist_paw = 0
    for nei in neigList:  
      try:
        temp_dist_imr = (centroid[0] - G.nodes[nei]['capability'][0])**2
        temp_dist_mmr = (centroid[1] - G.nodes[nei]['capability'][1])**2
        temp_dist_paw = (centroid[2] - G.nodes[nei]['capability'][2])**2
        temp_dist = math.sqrt(temp_dist_imr + temp_dist_mmr + temp_dist_paw)
        node_stress += temp_dist
      except(KeyError):
        pass  
    try:
      G.nodes[n]['nodeStress'] = node_stress/len(neigList)
    except(KeyError):
      # CHANGE THIS, IT IS WRONG
      G.nodes[n]['nodeStress'] = node_stress
      # G.nodes[n]["nodeStress"]  = math.sqrt((l2_sum_of_sq/len(neigList))**2)
    try:
      taluk_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
    except(KeyError):
      pass
  return taluk_stress_dict

In [10]:
for n in G.nodes():
    print(G.nodes[n])

{'capability': [0.898928343797887, 0.87420905051755, 0.8115493994576691], 'nodeStress': 0, 'name': 'Chikkodi'}
{'capability': [0.928773213688385, 0.367852027418498, 0.7679124211438318], 'nodeStress': 0, 'name': 'Athani'}
{'capability': [0.950744939682372, 0.788705322722176, 0.4868460240246204], 'nodeStress': 0, 'name': 'Raibag'}
{'capability': [0.977824170310372, 0.916406233132942, 0.8239307599974729], 'nodeStress': 0, 'name': 'Gokak'}
{'capability': [0.938356551853054, 0.788705322722176, 0.3482817853908891], 'nodeStress': 0, 'name': 'Hukkeri'}
{'capability': [0.646802379224394, 0.736032338613773, 0.8235381713491513], 'nodeStress': 0, 'name': 'Belagavi'}
{'capability': [0.235220240855489, 0.788705322722176, 0.7493389168163318], 'nodeStress': 0, 'name': 'Khanapur'}
{'capability': [0.913607182015361, 0.782658690455312, 0.8967163086094884], 'nodeStress': 0, 'name': 'Savadatti'}
{'capability': [0.587862963192241, 0.827531533727017, 0.8177116963521662], 'nodeStress': 0, 'name': 'Bailhongal'

In [11]:
stress = {}
for key_IMR, key_MMR, key_PAW in zip(dictionary_IMR.keys(), dictionary_MMR.keys(), dictionary_PAW()):    
    init_graph_attr(G,'IMR_Stress_AdjFile.xlsx',df_IMR, df_MMR, df_PAW, key_IMR, key_MMR, key_PAW)
    stress=get_node_stress(G)
    print(stress)
    df_IMR[dictionary_IMR[key_IMR]]=df_IMR['Taluka'].map(stress)
    df_MMR[dictionary_IMR[key_MMR]]=df_MMR['Taluka'].map(stress)
    df_PAW[dictionary_IMR[key_PAW]]=df_PAW['Taluka'].map(stress)

TypeError: 'dict' object is not callable

In [12]:
for n in G.nodes():
    print(G.nodes[n])

{'capability': [0.898928343797887, 0.87420905051755, 0.8115493994576691], 'nodeStress': 0, 'name': 'Chikkodi'}
{'capability': [0.928773213688385, 0.367852027418498, 0.7679124211438318], 'nodeStress': 0, 'name': 'Athani'}
{'capability': [0.950744939682372, 0.788705322722176, 0.4868460240246204], 'nodeStress': 0, 'name': 'Raibag'}
{'capability': [0.977824170310372, 0.916406233132942, 0.8239307599974729], 'nodeStress': 0, 'name': 'Gokak'}
{'capability': [0.938356551853054, 0.788705322722176, 0.3482817853908891], 'nodeStress': 0, 'name': 'Hukkeri'}
{'capability': [0.646802379224394, 0.736032338613773, 0.8235381713491513], 'nodeStress': 0, 'name': 'Belagavi'}
{'capability': [0.235220240855489, 0.788705322722176, 0.7493389168163318], 'nodeStress': 0, 'name': 'Khanapur'}
{'capability': [0.913607182015361, 0.782658690455312, 0.8967163086094884], 'nodeStress': 0, 'name': 'Savadatti'}
{'capability': [0.587862963192241, 0.827531533727017, 0.8177116963521662], 'nodeStress': 0, 'name': 'Bailhongal'

In [13]:
df_IMR.to_excel("3dNewStress_PIA_New_IMR.xlsx")
df_MMR.to_excel("3dNewStress_PIA_New_MMR.xlsx")
df_MMR.to_excel("3dNewStress_PIA_New_PAW.xlsx")

,District_GIS,Taluka,Infant Mortality Rate,Normalized IMR,Mothers who haven't had at least 4 antenatal care visits(%),Predicted Change In IMR (ANC -20%),New IMR (ANC-20%),Normalized IMR (ANC-20%),Impact Score (ANC -20%),Predicted Change In IMR (ANC -10%),...,Normalized IMR (Vitamin A-10%),Impact Score (Vitamin A -10%),Predicted Change In IMR (Vitamin A +10%),New IMR (Vitamin A +10%),Normalized IMR (Vitamin A+10%),Impact Score (Vitamin A +10%),Predicted Change In IMR (Vitamin A +20%),New IMR (Vitamin A+20%),Normalized IMR (Vitamin A+20%),Impact Score (Vitamin A +20%)
0,Kalburgi,Afzalpur,5.558428,0.942582,20.00355,-1.440256,4.118172,0.944348,0.490016,-0.720128,...,0.937551,0.326998,1.458078,7.016505,0.940914,0.673002,2.916155,8.474583,0.936699,0.673002
1,Chikkamagaluru,Ajjampura,13.672259,0.858767,15.92314,-1.146466,12.525793,0.858248,0.387197,-0.573233,...,0.852841,0.229948,1.284596,14.956855,0.858609,0.770052,2.569192,16.241451,0.856133,0.770052
2,Kalburgi,Aland,10.468411,0.891863,27.93242,-2.011134,8.457277,0.899912,0.689810,-1.005567,...,0.884162,0.151821,1.144939,11.613350,0.893266,0.848179,2.289878,12.758289,0.892264,0.848179
3,Dharwad,Alnavara,39.509821,0.591869,15.17857,-1.092857,38.416964,0.593104,0.368436,-0.546429,...,0.593815,0.507839,1.781343,41.291164,0.585644,0.492161,3.562686,43.072507,0.577813,0.492161
4,Hassan,Alur,36.381576,0.624183,10.93930,-0.787630,35.593947,0.622014,0.261613,-0.393815,...,0.621127,0.252618,1.325119,37.706695,0.622799,0.747382,2.650238,39.031815,0.619727,0.747382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Kalburgi,Yadrami,33.755352,0.651312,29.57785,-2.129605,31.625746,0.662651,0.731272,-1.064803,...,0.655061,0.640626,2.018708,35.774059,0.642831,0.359374,4.037415,37.792767,0.632580,0.359374
222,Chamarajanagara,Yalandur,18.631343,0.807541,14.56877,-1.048951,17.582391,0.806465,0.353070,-0.524476,...,0.808663,0.586384,1.921747,20.553090,0.800602,0.413616,3.843494,22.474837,0.791474,0.413616
223,Bengaluru_Urban,Yelahanka,45.156309,0.533541,3.09884,-0.223116,44.933192,0.526373,0.064048,-0.111558,...,0.537831,0.602668,1.950856,47.107165,0.525359,0.397332,3.901712,49.058021,0.515725,0.397332
224,Koppal,Yelburga,26.639909,0.724813,35.73127,-2.572651,24.067257,0.740055,0.886327,-1.286326,...,0.731425,0.839393,2.374016,29.013925,0.712902,0.160607,4.748033,31.387942,0.699018,0.160607
